In [1]:
import nltk; nltk.download('stopwords')

from nltk import sent_tokenize

from nltk.corpus import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

# import packages
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

import warnings

warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# # spacy for lemmatization
# import spacy      # May need to import spacy in command prompt first.
# import en_core_web_sm

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
#import matplotlib.pyplot as plt
#%matplotlib inline

# Enable logging for gensim - optional
import logging

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
df=pd.read_csv('stay.csv',encoding = 'unicode_escape')

In [3]:

text=df.Message.tolist()
c=[]
for m in text:
    m=m.split(' ')
   # while '@' in m:
    #    index=m.index('@')
     #   del m[index]
      #  if index+1<=len(m):
       #     del m[index]
    while '#' in m:
        index=m.index('#')
        del m[index]
        if index+1<=len(m):
            del m[index]
    m=' '.join(m)
    m=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', m)
    c.append(m)
all_docs1 = [nltk.word_tokenize(d) for d in c]
all_docs2 = [[w.lower() for w in doc] for doc in all_docs1]
all_docs3 = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in all_docs2]
stop_list = stopwords.words('english')

clean_text = [[w for w in doc if w not in stop_list] for doc in all_docs3]
from nltk.stem.snowball import *
stemmer = SnowballStemmer('english')
clean_text1 = [[stemmer.stem(word) for word in w] for w in clean_text]


In [4]:

dictionary = corpora.Dictionary(clean_text1)
# print(dictionary)
corpus = [dictionary.doc2bow(doc) for doc in clean_text1]

In [5]:
tfidf = gensim.models.tfidfmodel.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
dict1 = {}
for doc in corpus_tfidf:
    for id, value in doc:
        word = dictionary.get(id)
        dict1[word] = value

print(len(dict1))


df2 = pd.DataFrame(list(dict1.items()),columns=['word', 'IDF'])
df2 = df2.sort_values(by=['IDF'], ascending= False)

print(len(df2))

df3 = df2[np.logical_or(df2['IDF'] > 0.8,df2['IDF'] < 0.05)]

print(len(df3))


#writer = pd.ExcelWriter("/Users/cjy/Desktop/Social/leaveidf2.xlsx")
#df3.to_excel(writer, sheet_name='Sheet1')
#writer.save()
#writer.close()

9743
9743
390


In [6]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=30,
                                           passes=100,
                                           alpha='auto')

In [7]:
doc_lda = lda_model[corpus]

lda_model.show_topics(num_topics=3,num_words=5)

[(14,
  '0.108*"plan" + 0.064*"stop" + 0.051*"thought" + 0.043*"absolut" + 0.042*"interest"'),
 (15,
  '0.278*"vote" + 0.124*"remain" + 0.072*"need" + 0.070*"britain" + 0.059*"europ"'),
 (18,
  '0.076*"peopl" + 0.073*"want" + 0.059*"us" + 0.057*"get" + 0.057*"would"')]

In [8]:
import pyLDAvis.gensim

In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

C:\Users\80600\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18    -0.385430  0.239699       1        1  8.353068
15    -0.149883 -0.351920       2        1  5.653442
8     -0.121827 -0.154658       3        1  5.426393
3     -0.053291 -0.032750       4        1  4.576020
25    -0.055287 -0.035815       5        1  4.387431
27    -0.030164 -0.017485       6        1  3.806036
7     -0.014096 -0.005621       7        1  3.736558
2     -0.014841 -0.005226       8        1  3.732796
26    -0.011971 -0.004326       9        1  3.695860
1      0.017034  0.009970      10        1  3.434766
6      0.022190  0.012955      11        1  3.326743
17     0.013335  0.012280      12        1  3.298670
13     0.004246  0.003086      13        1  3.287276
10     0.019977  0.011086      14        1  3.110732
4      0.024307  0.012372      15        1  3.041003
23     0.008521  0.021144      16        1  2.943351
20     0.039184  0.019425      17        1  2.794995
21     0.038921  0.016790      18        1  2.777037
28     0.043020  0.019436      19        1  2.737930
12     0.038841  0.017430      20        1  2.673478
9      0.052101  0.021942      21        1  2.562122
24     0.045774  0.019759      22        1  2.506034
29     0.054014  0.022283      23        1  2.421716
5      0.059520  0.023054      24        1  2.355011
16     0.052418  0.023018      25        1  2.331457
22     0.055603  0.017629      26        1  2.311243
19     0.058739  0.023697      27        1  2.295018
11     0.054703  0.022169      28        1  2.283002
0      0.070785  0.028607      29        1  2.103366
14     0.063554  0.009972      30        1  2.037453, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
59    Default  2299.000000  strongerin  2299.000000  30.0000  30.0000
99    Default  2021.000000          eu  2021.000000  29.0000  29.0000
8     Default  1719.000000        vote  1719.000000  28.0000  28.0000
1     Default  1311.000000        leav  1311.000000  27.0000  27.0000
24    Default   690.000000         say   690.000000  26.0000  26.0000
228   Default   766.000000          uk   766.000000  25.0000  25.0000
80    Default   765.000000      remain   765.000000  24.0000  24.0000
97    Default   565.000000    campaign   565.000000  23.0000  23.0000
264   Default   695.000000       peopl   695.000000  22.0000  22.0000
82    Default   476.000000        back   476.000000  21.0000  21.0000
14    Default   493.000000        like   493.000000  20.0000  20.0000
169   Default   667.000000        want   667.000000  19.0000  19.0000
121   Default   459.000000          go   459.000000  18.0000  18.0000
148   Default   431.000000        know   431.000000  17.0000  17.0000
30    Default   442.000000        bori   442.000000  16.0000  16.0000
20    Default   395.000000      brexit   395.000000  15.0000  15.0000
195   Default   397.000000        take   397.000000  14.0000  14.0000
189   Default   417.000000         one   417.000000  13.0000  13.0000
367   Default   380.000000         lie   380.000000  12.0000  12.0000
234   Default   403.000000       right   403.000000  11.0000  11.0000
74    Default   538.000000          us   538.000000  10.0000  10.0000
192   Default   366.000000     control   366.000000   9.0000   9.0000
78    Default   522.000000         get   522.000000   8.0000   8.0000
176   Default   519.000000       would   519.000000   7.0000   7.0000
318   Default   446.000000        need   446.000000   6.0000   6.0000
72    Default   506.000000       think   506.000000   5.0000   5.0000
214   Default   433.000000     britain   433.000000   4.0000   4.0000
232   Default   372.000000      immigr   372.000000   3.0000   3.0000
183   Default   481.000000        make   481.000000   2.0000   2.0000
315   Default   305.000000      better   305.000000   1.0000   1.0000
...       ...        

In [10]:
pyLDAvis.save_html(vis, 'stay.html')